In [1]:
import h5py
import mlflow
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import DataLoader

from autoencoder.dataset import MRIMemorySHDataset
from autoencoder.spherical.CG import real_clebsch_gordan_all
from autoencoder.spherical.convolution import S2Convolution, SO3Convolution

In [2]:
latent_features = np.loadtxt("latent_features.txt", dtype=int)

batch_size = 256
train_subjects = [11, 12, 13, 14]
validate_subjects = [15]

train_dataset = MRIMemorySHDataset(
    "../data/data.hdf5",
    train_subjects,
    include=latent_features,
    l_bandwidth=[0] * 5,
)

validate_dataset = MRIMemorySHDataset(
    "../data/data.hdf5",
    validate_subjects,
    include=latent_features,
    l_bandwidth=[0] * 5,
)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    pin_memory=True,
    batch_size=batch_size,
    num_workers=6,
)
validate_dataloader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=6)

/home/maarten/Workspace/School/uu/thesis/geometric-dl-dmri/src/autoencoder/spherical/harmonics.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  Y[:, n_sph : n_sph + 2 * i + 1] = Y_n_m


In [4]:
with h5py.File("../data/data.hdf5", "r") as archive:
    scheme = archive.get("scheme")[()]

rng = np.random.default_rng()
latent_features = rng.choice(1344, 500, replace=False)

In [4]:
latent_features = np.loadtxt("latent_features.txt", dtype=int)

In [ ]:
latent_scheme = scheme[latent_features]
df = pd.DataFrame(latent_scheme).groupby([4, 5], as_index=False).count().reset_index()

df[["index", 0]].values

In [3]:
class SphericalDecoder(pl.LightningModule):
    def __init__(self, *, learning_rate: float = 1e-3, profiler=None) -> None:
        super(SphericalDecoder, self).__init__()

        self.learning_rate = learning_rate

        L = [0] * 4
        CG_r, CG_l = real_clebsch_gordan_all(L[0], L[1], device="cuda")

        self.s2_conv = S2Convolution(
            28, 3, L[0], L[1], 6, 8, CG_r, CG_l, profiler=profiler
        )
        self.so3_conv1 = SO3Convolution(
            28, 3, L[1], L[2], 8, 16, CG_r, CG_l, profiler=profiler
        )

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(2016, 1500),
            torch.nn.Linear(1500, 1344),
        )

    def forward(self, x: dict[int, torch.Tensor]) -> torch.Tensor:
        rh0, feats0 = self.s2_conv(x)
        rh1, feats1 = self.so3_conv1(rh0)

        features = torch.cat((feats0, feats1), dim=1)

        decoded = self.linear(features)

        return decoded

    def configure_optimizers(self) -> torch.optim.Adam:
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(
        self,
        batch: dict[str, torch.Tensor],
        batch_idx: int,
    ) -> torch.Tensor:
        return self._shared_eval(batch, batch_idx, "train")

    def validation_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        return self._shared_eval(batch, batch_idx, "val")

    def _shared_eval(
        self, batch: torch.Tensor, batch_idx: int, prefix: str
    ) -> torch.Tensor:
        """Calculate the loss for a batch.

        Args:
            batch (torch.Tensor): batch data.
            batch_idx (int): batch id.
            prefix (str): prefix for logging.

        Returns:
            torch.Tensor: calculated loss.
        """
        data, target = batch["data"], batch["target"]

        decoded = self.forward(data)
        loss = F.mse_loss(decoded, target)

        self.log(f"{prefix}_loss", loss, on_step=False, on_epoch=True, prog_bar=True)

        return loss

In [ ]:
experiment_name = "spherical_decoder"

# mlflow.set_tracking_uri(os.environ["MLFLOW_ENDPOINT_URL"])

mlflow.set_experiment(experiment_name)
mlflow.pytorch.autolog()

profiler = pl.profiler.AdvancedProfiler(filename="test_sh.txt")
profiler = None

model = SphericalDecoder(profiler=profiler)
trainer = pl.Trainer(
    gpus=-1,
    profiler=profiler,
    max_epochs=2000,
    callbacks=[
        EarlyStopping(monitor="val_loss"),
        ModelCheckpoint(monitor="val_loss"),
    ],
)
trainer.fit(model, train_dataloader, validate_dataloader)

2021/12/13 15:12:32 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
2021/12/13 15:12:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '58d04b12edee48919c9e16c0e7f2b7c1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | s2_conv   | S2Convolution  | 4.0 K 
1 | so3_conv1 | SO3Convolution | 12.1 K
2 | linear    | Sequential     | 5.0 M 
---------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.236    Total 

Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/home/maarten/miniconda3/envs/mudi/lib/python3.9/site-packages/pytorch_lightning/trainer/callback_hook.py:102: LightningDeprecationWarning: The signature of `Callback.on_train_epoch_end` has changed in v1.3. `outputs` parameter has been removed. Support for the old signature will be removed in v1.5
  warning_cache.deprecation(
/home/maarten/miniconda3/envs/mudi/lib/python3.9/site-packages/pytorch_lightning/trainer/

...

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

Validating: 0it [00:00, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

Validating: 0it [00:00, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn